In [1]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import iirnotch, lfilter
from notch_filter_cls import notch_filter_cls  # Assuming notch_filter_cls function is in notch_filter_cls.py

Qfactor = 45.0 # standard IIR notch filter's Q-factor
lambda_ = 25000.0 # notch_filter_cls's regularization factor (the higher, the sharper the notch frequency is)
bias = 0  # offsets between plots; adjust as needed

# File paths
rawdatafilepath = '/Users/rsameni/Documents/GitHub/zero-transition-notch-filter/sample-data/LAMainsNoiseExamples' #'../sample-data/LAMainsNoiseExamples/'
outputdatafilepath = './results/'

# Process all data files
all_fnames = [f for f in os.listdir(rawdatafilepath) if f.endswith('.json')]

for fname in all_fnames:
    filepath = os.path.join(rawdatafilepath, fname)
    with open(filepath, 'r') as fid:
        data = json.load(fid)

    fmains = data['mainsFreq']
    fs = data['rawEcg']['samplesPerSecond']
    fs_alivecor_filt = data['enhEcg']['samplesPerSecond']

    for ch, x_raw in data['rawEcg']['ecgSamples'].items():
        x_raw = np.array(x_raw) - np.mean(x_raw)
        x_alivecor_filt = np.array(data['enhEcg']['ecgSamples'][ch])

        # Truncate signals
        x_raw = x_raw[:-3]
        x_alivecor_filt = x_alivecor_filt[:-3]

        # Check if x_raw is 1D or 2D and process accordingly
        x_raw_reshaped = x_raw.reshape(1, -1) if x_raw.ndim == 1 else x_raw

        # Custom notch filter
        x_cls_reshaped = notch_filter_cls(x_raw_reshaped, fmains/fs, lambda_)
        x_cls = x_cls_reshaped.ravel() if x_raw.ndim == 1 else x_cls_reshaped

        # Standard IIR notch filter
        b, a = iirnotch(fmains, Qfactor, fs)
        x_iir = lfilter(b, a, x_raw)

        # Plotting
        t_raw = np.arange(len(x_raw)) / fs
        t_alivecor_filt = np.arange(len(x_alivecor_filt)) / fs_alivecor_filt
        plt.figure(figsize=(18, 6))
        plt.plot(t_raw, x_raw, label='Raw ECG')
        plt.plot(t_raw, x_iir - 3*bias, label='IIR Filtered ECG')
        plt.plot(t_raw, x_cls - 2*bias, label='Custom Filtered ECG')
        plt.plot(t_alivecor_filt, x_alivecor_filt - np.mean(x_alivecor_filt) - bias, label='AliveCor Filtered ECG')
        plt.grid()
        plt.legend()
        plt.xlabel('Time (seconds)', fontsize=14)
        plt.ylabel('Amplitude (uV)', fontsize=14)
        plt.title(f"{fname}, Channel: {ch}", fontsize=14)
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.savefig(os.path.join(outputdatafilepath, f"{fname[:-5]}_{ch}_filtered.png"))
        plt.close()

print('Done!')


Done!
